In [240]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
%matplotlib inline 

In [241]:
train_df = pd.read_csv('train.csv',index_col=0)
test_df = pd.read_csv('test.csv',index_col=0)
col_names = train_df.columns.tolist()

print "Column names:"
print col_names

print "\nSample data:"
train_df.head()

Column names:
['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']

Sample data:


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [242]:
encode_features = ['Gender','Married','Education','Self_Employed','Dependents','Loan_Status']

fillna_withmean = ['LoanAmount','Loan_Amount_Term']
fillna_withmostcommon = ['Dependents','Gender','Credit_History','Married','Self_Employed']


def transform_df(data):
    
    #Removing Loans_ID
    df = data #.drop('Loan_ID',axis=1) 
    
    # Filling NaN values 
    for feature in fillna_withmean:
        if feature in data.columns.values:
            df[feature] = df[feature].fillna(df[feature].mean()) 
        
    for feature in fillna_withmostcommon:
        if feature in data.columns.values:
            df[feature] = df[feature].fillna(df[feature].value_counts().index[0])
    
    # Encoding Features
    for feature in encode_features:
        if feature in data.columns.values:
            le = LabelEncoder()
            df[feature] = le.fit_transform(df[feature])
    
    # Adding Applicant and Coapplicant Incomes as Household
    df['Household_Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']
    df = df.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)
     
    # Transforming some other values   
    dummies = pd.get_dummies(df.Property_Area)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop('Property_Area',axis=1)

    
    return df

train_df = transform_df(train_df)
test_df = transform_df(test_df)


In [244]:
train_df.insert(len(train_df.columns)-1,'Loan_Status',train_df.pop('Loan_Status'))

In [245]:
scale_features = ['LoanAmount','Loan_Amount_Term','Household_Income']

train_df[scale_features] = train_df[scale_features].apply(lambda x:(x.astype(int) - min(x))/(max(x)-min(x)), axis = 0)
test_df[scale_features] = test_df[scale_features].apply(lambda x:(x.astype(int) - min(x))/(max(x)-min(x)), axis = 0)

In [246]:
X_train = train_df.iloc[:, :-1]
Y_train = train_df.iloc[:,-1]
X_test = test_df

In [247]:
Y_train.head()

Loan_ID
LP001002    1
LP001003    0
LP001005    1
LP001006    1
LP001008    1
Name: Loan_Status, dtype: int64

In [248]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf = clf.fit(X_train, Y_train)

Y_test = clf.predict(X_test)


In [249]:
Y_test = ['Y' if x==1 else 'N' for x in Y_test]

In [250]:
X_test['Loan_Status']=Y_test
X_test = X_test['Loan_Status']

In [251]:
X_test.to_csv('loans_submission.csv',sep=',',header=True)